All of these libraries should import.

In [2]:
import os
from os import listdir
from os.path import isfile, join

os.environ['KERAS_BACKEND'] = 'tensorflow'
#os.environ['KERAS_BACKEND'] = 'theano'

import random
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import mobilenet
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

This helper function will load and pre-process the image, and return it and the pre-processed pixel vector.

In [3]:
def get_image(path, w, h):
    img = image.load_img(path, target_size=(w, h))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x


We'll set our image width and height to be 224 x 224.

In [4]:
w, h = 224, 224

Let's load all the data. We'll use a three-class classifier as an example from 101ObjectCategories: 

In [5]:
categories = ["airplanes", "crocodile", "dragonfly"]
data = []
for c, category in enumerate(categories):
    path = '101_ObjectCategories/%s' % category
    images = [f for f in listdir(path) if isfile(join(path, f))]
    for img_path in images:
        img, x = get_image(join(path, img_path), w, h)
        data.append({'x':np.array(x[0]), 'y':c})
        
num_classes = len(categories)

We split the data into training set, validation set, and test set (70%, 15%, 15%). We also pre-process the data by normalizing it and casting to float32. We 

In [6]:
train_split, val_split = 0.7, 0.15


# create training / validation / test split (70%, 15%, 15%)
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

# separate data for labels
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]

# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)


finished loading 918 images from 3 categories
train / validation / test split: 642, 138, 138
training data shape:  (642, 224, 224, 3)
training labels shape:  (642, 3)


Load mobilenet model.

In [7]:
def loadModel():
    mobilenet = keras.applications.mobilenet.MobileNet()
    model = Model(inputs=mobilenet.input,outputs=mobilenet.get_layer('conv_pw_13_relu').output)
    return model

mobilenet = loadModel()
mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

Add classification layer to the top and freeze the other layers.

In [22]:
# make a new softmax layer with num_classes neurons
flatten = Flatten(input_shape=(7,7,1024))(mobilenet.layers[-1].output)
fc = Dense(100, activation='relu')(flatten)
output = Dense(num_classes, activation='softmax')(fc)

model = Model(mobilenet.input, output)


# make all layers untrainable by freezing weights (except for last two layers)
for l, layer in enumerate(model.layers[:-2]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model.layers[-2:]):
    layer.trainable = True

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

Let's train:

In [23]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=16)


Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[50176,1000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_11/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=339866, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_11/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_11/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/gene/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in l

In [6]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

('Test loss:', 0.280648006429334)
('Test accuracy:', 0.952380947650425)


In [29]:

import time



t1 = time.time()
idx = int(50*random.random())
y_prob = model.predict(x_test[idx].reshape((1,224,224,3)))
actual = np.argmax(y_test[idx])
prediction = np.argmax(y_prob)
print("predicted %d (actual %d) in %0.3f" % (prediction, actual, (time.time()-t1)))


predicted 1 (actual 1) in 0.219


In [27]:
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# loads a compiled model 
#model = load_model('my_model.h5')

In [80]:
for p, a in zip(y_pred, y_test):
    print(np.argmax(p), np.argmax(a), np.abs(np.argmax(p)-np.argmax(a)))

(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 1, 1)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(1, 0, 1)
(1, 1, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(0, 1, 1)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(0, 0, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(1, 1, 0)
(0, 0, 0)
(1, 1, 0)
(1, 1, 0)
